In [29]:
import json
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Define data folder
data_folder = '/content/Data'
output_folder = '/content/Output'

if not os.path.exists(data_folder):
    print("Data folder not found.")
os.makedirs(output_folder, exist_ok=True)
session_records = []

for filename in os.listdir(data_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(data_folder, filename)
        with open(file_path, 'r') as file:
            player_data = json.load(file)
            for session in player_data['sessions']:
                session_record = session.copy()

                target_labels = {
                    "avgCompetence": round(session.get('avgCompetence', 0)),
                    "avgFlow": round(session.get('avgFlow', 0)),
                    "avgTension": round(session.get('avgTension', 0)),
                    "avgChallenge": round(session.get('avgChallenge', 0)),
                    "avgNegativeAffect": round(session.get('avgNegativeAffect', 0)),
                    "avgPositiveAffect": round(session.get('avgPositiveAffect', 0))
                }
                session_record.update(target_labels)

                normalization_factor = session.get('timesRestarted', 0) + 1
                for key, value in session.items():
                    if key not in target_labels and key not in ['timesRestarted', 'levelComplete']:
                        session_record[key] = value / normalization_factor

                walk_speed = 5
                sprint_speed = 10
                time_sprinting = session.get('timeSprinting', 0)
                time_walking = session.get('timeWalking', 0)
                total_distance = (time_sprinting * sprint_speed) + (time_walking * walk_speed)
                session_record['averageSpeed'] = total_distance / session.get('levelTime', 1)

                session_records.append(session_record)

df = pd.DataFrame(session_records)
target_columns = ['avgCompetence', 'avgFlow', 'avgTension', 'avgChallenge', 'avgNegativeAffect', 'avgPositiveAffect']
X = df.drop(columns=target_columns)
y = df[target_columns]

# Standardization
features_to_scale = ['levelTime', 'timesRestarted', 'timesDied', 'timeSprinting', 'timeWalking',
                'damageTakenNormal', 'damageTakenBomb', 'damageTakenBullet',
                'enemyJumpsMissed', 'enemyBulletsMissed', 'enemyBombsMissed',
                'bombsIgnited', 'timesPaused', 'detectionTime', 'collisions']
scaler = StandardScaler()
X[features_to_scale] = scaler.fit_transform(X[features_to_scale])

In [30]:
# Final Model Training (80/20 Split)
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2)

final_model = Sequential([
    Dense(128, activation=LeakyReLU(), input_shape=(X_train_final.shape[1],), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation=LeakyReLU(), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(32, activation=LeakyReLU(), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dense(y_train_final.shape[1])
])

final_model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=1.0), metrics=['mae'])
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history_final = final_model.fit(X_train_final, y_train_final, epochs=150, validation_data=(X_test_final, y_test_final), callbacks=[early_stopping])
#history_final = final_model.fit(X_train_final, y_train_final, epochs=150, validation_data=(X_test_final, y_test_final))

y_final_pred = np.clip(np.round(final_model.predict(X_test_final)), 1, 5)

# Calculate accuracy and save results
rmses = []  # To store RMSE for each target
with open(os.path.join(output_folder, 'final_model_results.txt'), 'w') as rmse_results_file:
    for i, column in enumerate(target_columns):
        actual_values = y_test_final[column].values
        predicted_values = y_final_pred[:, i]

        # Calculate Root Mean Squared Error
        mse = np.mean((actual_values - predicted_values) ** 2)
        rmse = np.sqrt(mse)
        rmses.append(rmse)

        # Calculate Percentage Accuracy based on RMSE
        max_error = 4  # Maximum error for range 1 to 5
        percentage_accuracy = (1 - (rmse / max_error)) * 100

        # Write RMSE and Percentage Accuracy result for each target to file
        rmse_results_file.write(f'Root Mean Squared Error for {column}: {rmse:.2f}\n')
        rmse_results_file.write(f'Percentage Accuracy for {column}: {percentage_accuracy:.2f}%\n')
        print(f'Root Mean Squared Error for {column}: {rmse:.2f}')
        print(f'Percentage Accuracy for {column}: {percentage_accuracy:.2f}%')

    # Calculate and write overall RMSE and Percentage Accuracy
    overall_rmse = np.mean(rmses)
    overall_percentage_accuracy = (1 - (overall_rmse / max_error)) * 100
    rmse_results_file.write(f'Overall Root Mean Squared Error: {overall_rmse:.2f}\n')
    rmse_results_file.write(f'Overall Percentage Accuracy: {overall_percentage_accuracy:.2f}%\n')
    print(f'Overall Root Mean Squared Error: {overall_rmse:.2f}')
    print(f'Overall Percentage Accuracy: {overall_percentage_accuracy:.2f}%\n')

# Final Model Visualization
for i, column in enumerate(target_columns):
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=y_test_final[column], y=y_final_pred[:, i], color='blue')
    plt.plot([1, 5], [1, 5], 'r--')
    plt.title(f'Final Model: Actual vs Predicted for {column}')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.xlim(1, 5)
    plt.ylim(1, 5)
    plt.grid()
    plt.savefig(os.path.join(output_folder, f'final_model_actual_vs_predicted_{column}.png'))
    plt.close()

# Final Loss and MAE Plots
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_final.history['loss'], label='Training Loss')
plt.plot(history_final.history['val_loss'], label='Validation Loss')
plt.title('Final Model - Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history_final.history['mae'], label='Training MAE')
plt.plot(history_final.history['val_mae'], label='Validation MAE')
plt.title('Final Model - MAE')
plt.ylabel('Mean Absolute Error')
plt.xlabel('Epoch')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'final_model_loss_mae.png'))
plt.close()

# Save Final Model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_model = converter.convert()
with open(os.path.join(output_folder, 'final_model.tflite'), 'wb') as f:
    f.write(tflite_model)

print("Model training complete and saved as 'final_model.tflite'")

Epoch 1/150


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - loss: 2.7379 - mae: 3.0636 - val_loss: 2.9241 - val_mae: 3.2448
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.7551 - mae: 3.0785 - val_loss: 2.9288 - val_mae: 3.2521
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 2.7973 - mae: 3.1285 - val_loss: 2.9371 - val_mae: 3.2618
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.7538 - mae: 3.0763 - val_loss: 2.9440 - val_mae: 3.2703
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.7390 - mae: 3.0653 - val_loss: 2.9526 - val_mae: 3.2798
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.7309 - mae: 3.0555 - val_loss: 2.9521 - val_mae: 3.2805
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.7436 - mae: 3.0659 - val_loss: 2.9529 - val_mae: 3.2831
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.7119 - mae: 3.0400 - val_loss: 2.9574 - val_mae: 3.2896
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.7041 - mae: 3.0356 - val_l

In [38]:
output_folder_models = "Models"
os.makedirs(output_folder, exist_ok=True)

# Save model in TensorFlow SavedModel format
saved_model_path = os.path.join(output_folder_models, "final_model_saved")
final_model.export(saved_model_path)
print(f"Model saved in SavedModel format at '{saved_model_path}'")

# Save model in Keras HDF5 format
h5_model_path = os.path.join(output_folder_models, "final_model.h5")
final_model.save(h5_model_path)
print(f"Model saved as Keras .h5 format at '{h5_model_path}'")

# Save model in Keras .keras format
keras_model_path = os.path.join(output_folder_models, "final_model.keras")
final_model.save(keras_model_path)
print(f"Model saved as Keras .keras format at '{keras_model_path}'")

# Save Final Model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_model = converter.convert()
tflite_model_path = os.path.join(output_folder_models, 'final_model.tflite')
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print(f"Model converted to TensorFlow Lite format and saved at '{tflite_model_path}'")

Saved artifact at 'Models/final_model_saved'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 17), dtype=tf.float32, name='keras_tensor_358')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  140703346752528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350288128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350276864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350285840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350291296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350279504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350287072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350281088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350282848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350285136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14070335028

Model saved in SavedModel format at 'Models/final_model_saved'
Model saved as Keras .h5 format at 'Models/final_model.h5'
Model saved as Keras .keras format at 'Models/final_model.keras'
Saved artifact at '/tmp/tmp02jcm4va'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 17), dtype=tf.float32, name='keras_tensor_358')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  140703346752528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350288128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350276864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350285840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350291296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350279504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350287072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140703350281088: TensorSpec(shape=(), dtyp

In [42]:
!pip install tf2onnx
import tf2onnx

onnx_model_path = os.path.join(output_folder_models, "final_model.onnx")
# Convert from SavedModel to ONNX
!python -m tf2onnx.convert --saved-model ./Models/final_model_saved --output ./Models/final_model.onnx --opset 13
print(f"Model converted to ONNX format and saved at '{onnx_model_path}'")

!pip install onnxruntime
import onnx
import onnxruntime as ort

# Load and check the ONNX model
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

# Run inference with ONNX Runtime
ort_session = ort.InferenceSession(onnx_model_path)
print("ONNX model loaded and verified successfully.")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
I0000 00:00:1730650746.304460   92606 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730650746.360606   92606 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730650746.360890   92606 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA n

In [34]:
# K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True)
overall_accuracies = {column: [] for column in target_columns}
model_results = []

for fold, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Model Setup
    model = Sequential([
        Dense(128, activation=LeakyReLU(), input_shape=(X_train_final.shape[1],), kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64, activation=LeakyReLU(), kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(32, activation=LeakyReLU(), kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dense(y_train_final.shape[1])
    ])

    model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=1.0), metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

    # Train Model
    history = model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), callbacks=[early_stopping])
    #history = model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test))

    # Predictions
    y_pred = np.clip(np.round(model.predict(X_test)), 1, 5)

    # Calculate RMSE and Percentage Accuracy
    rmses = []  # To store RMSE for each target
    percentage_accuracies = []  # To store percentage accuracy for each target
    max_error = 4  # Maximum error for range 1 to 5
    print(f'Fold {fold + 1}:')
    for i, column in enumerate(target_columns):
        actual_values = y_test[column].values
        predicted_values = y_pred[:, i]

        # Calculate RMSE
        mse = np.mean((actual_values - predicted_values) ** 2)
        rmse = np.sqrt(mse)
        rmses.append(rmse)

        # Calculate Percentage Accuracy
        percentage_accuracy = (1 - (rmse / max_error)) * 100
        percentage_accuracies.append(percentage_accuracy)

        # Print Results for Each Target Variable
        print(f'Root Mean Squared Error for {column}: {rmse:.2f}')
        print(f'Percentage Accuracy for {column}: {percentage_accuracy:.2f}%')

    # Overall RMSE and Percentage Accuracy
    overall_rmse = np.mean(rmses)
    overall_percentage_accuracy = (1 - (overall_rmse / max_error)) * 100
    model_results.append({"fold": fold + 1, "rmses": rmses, "percentage_accuracies": percentage_accuracies})

    # Print Overall Results
    print(f'Overall Root Mean Squared Error: {overall_rmse:.2f}')
    print(f'Overall Percentage Accuracy: {overall_percentage_accuracy:.2f}%\n')

    # Visualization for Actual vs Predicted
    for i, column in enumerate(target_columns):
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=y_test[column], y=y_pred[:, i], color='blue')
        plt.plot([1, 5], [1, 5], 'r--')  # Perfect Prediction Line
        plt.title(f'Actual vs Predicted for {column} - Fold {fold + 1}')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.xlim(1, 5)
        plt.ylim(1, 5)
        plt.grid()
        plt.savefig(os.path.join(output_folder, f'fold_{fold + 1}_actual_vs_predicted_{column}.png'))
        plt.close()

    # Loss and MAE Plots
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {fold + 1} - Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Training MAE')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.title(f'Fold {fold + 1} - MAE')
    plt.ylabel('Mean Absolute Error')
    plt.xlabel('Epoch')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f'fold_{fold + 1}_loss_mae_fold.png'))
    plt.close()

# Save K-Fold Results with formatted output
with open(os.path.join(output_folder, 'kfold_model_results.txt'), 'w') as results_file:
    overall_rmses = np.zeros(len(target_columns))
    overall_percentage_accuracies = np.zeros(len(target_columns))

    for result in model_results:
        results_file.write(f"Fold {result['fold']}:\n")
        for i, column in enumerate(target_columns):
            rms = result["rmses"][i]
            percentage_acc = result["percentage_accuracies"][i]
            results_file.write(f'Root Mean Squared Error for {column}: {rms:.2f}\n')
            results_file.write(f'Percentage Accuracy for {column}: {percentage_acc:.2f}%\n')
            overall_rmses[i] += rms
            overall_percentage_accuracies[i] += percentage_acc

        overall_rmse_fold = np.mean(result["rmses"])
        overall_percentage_accuracy_fold = (1 - (overall_rmse_fold / max_error)) * 100
        results_file.write(f'Overall Root Mean Squared Error: {overall_rmse_fold:.2f}\n')
        results_file.write(f'Overall Percentage Accuracy: {overall_percentage_accuracy_fold:.2f}%\n\n')

    # Calculate overall metrics across all folds
    num_folds = len(model_results)
    results_file.write("Across all folds:\n")
    print("Across all folds:")
    for i, column in enumerate(target_columns):
        avg_rmse = overall_rmses[i] / num_folds
        results_file.write(f'Root Mean Squared Error for {column}: {avg_rmse:.2f}\n')
        print(f'Root Mean Squared Error for {column}: {avg_rmse:.2f}')
        avg_accuracy = overall_percentage_accuracies[i] / num_folds
        results_file.write(f'Percentage Accuracy for {column}: {avg_accuracy:.2f}%\n')
        print(f'Percentage Accuracy for {column}: {avg_accuracy:.2f}%')

    # Calculate overall RMSE and Percentage Accuracy for all folds
    overall_rmse_final = np.mean(overall_rmses / num_folds)
    overall_percentage_accuracy_final = (1 - (overall_rmse_final / max_error)) * 100
    results_file.write(f'Overall Root Mean Squared Error: {overall_rmse_final:.2f}\n')
    results_file.write(f'Overall Percentage Accuracy: {overall_percentage_accuracy_final:.2f}%\n')
    print(f'Overall Root Mean Squared Error: {overall_rmse_final:.2f}')
    print(f'Overall Percentage Accuracy: {overall_percentage_accuracy_final:.2f}%')

Epoch 1/150


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - loss: 2.8307 - mae: 3.1493 - val_loss: 2.5786 - val_mae: 2.8948
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2.8062 - mae: 3.1297 - val_loss: 2.5677 - val_mae: 2.8858
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.8153 - mae: 3.1446 - val_loss: 2.5614 - val_mae: 2.8813
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.8286 - mae: 3.1555 - val_loss: 2.5560 - val_mae: 2.8771
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7788 - mae: 3.1018 - val_loss: 2.5475 - val_mae: 2.8686
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.7667 - mae: 3.0939 - val_loss: 2.5387 - val_mae: 2.8597
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.7676 - mae: 3.0940 - val_loss: 2.5247 - val_mae: 2.8449
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.7417 - mae: 3.0628 - val_loss: 2.5099 - val_mae: 2.8287
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 2.7154 - mae: 3.0469 - val_l

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 2.7761 - mae: 3.0968 - val_loss: 2.7368 - val_mae: 3.0470
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 2.7850 - mae: 3.1041 - val_loss: 2.7307 - val_mae: 3.0415
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7482 - mae: 3.0708 - val_loss: 2.7299 - val_mae: 3.0410
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7304 - mae: 3.0533 - val_loss: 2.7262 - val_mae: 3.0375
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7421 - mae: 3.0683 - val_loss: 2.7217 - val_mae: 3.0330
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7255 - mae: 3.0472 - val_loss: 2.7147 - val_mae: 3.0256
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.7177 - mae: 3.0472 - val_loss: 2.7103 - val_mae: 3.0214
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.7238 - mae: 3.0518 - val_loss: 2.7064 - val_mae: 3.0203
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.6885 - mae: 3.

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - loss: 2.8227 - mae: 3.1456 - val_loss: 2.5621 - val_mae: 2.8988
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.7764 - mae: 3.1038 - val_loss: 2.5511 - val_mae: 2.8895
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 2.7582 - mae: 3.0803 - val_loss: 2.5483 - val_mae: 2.8881
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.7484 - mae: 3.0638 - val_loss: 2.5429 - val_mae: 2.8836
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 2.7659 - mae: 3.0953 - val_loss: 2.5425 - val_mae: 2.8839
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.6934 - mae: 3.0221 - val_loss: 2.5440 - val_mae: 2.8860
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.6771 - mae: 3.0011 - val_loss: 2.5426 - val_mae: 2.8850
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.6856 - mae: 3.0084 - val_loss: 2.5411 - val_mae: 2.8838
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.6734 - mae: 3.0018 - val_l

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 2.7912 - mae: 3.1018 - val_loss: 3.4096 - val_mae: 3.7497
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 2.7523 - mae: 3.0760 - val_loss: 3.3555 - val_mae: 3.6958
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.7149 - mae: 3.0425 - val_loss: 3.3074 - val_mae: 3.6479
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.7298 - mae: 3.0552 - val_loss: 3.2562 - val_mae: 3.5971
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 2.6959 - mae: 3.0231 - val_loss: 3.2104 - val_mae: 3.5516
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 2.6917 - mae: 3.0185 - val_loss: 3.1712 - val_mae: 3.5128
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.6861 - mae: 3.0117 - val_loss: 3.1358 - val_mae: 3.4777
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.6920 - mae: 3.0160 - val_loss: 3.1006 - val_mae: 3.4428
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.6990 - mae: 3.0303 - val_l

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - loss: 2.6658 - mae: 2.9731 - val_loss: 3.2342 - val_mae: 3.5736
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 2.7206 - mae: 3.0336 - val_loss: 3.2062 - val_mae: 3.5462
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 2.7094 - mae: 3.0269 - val_loss: 3.1844 - val_mae: 3.5246
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 2.6373 - mae: 2.9642 - val_loss: 3.1699 - val_mae: 3.5103
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 2.6615 - mae: 2.9814 - val_loss: 3.1570 - val_mae: 3.4976
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2.7070 - mae: 3.0306 - val_loss: 3.1464 - val_mae: 3.4873
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 2.6231 - mae: 2.9488 - val_loss: 3.1363 - val_mae: 3.4773
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.6132 - mae: 2.9485 - val_loss: 3.1245 - val_mae: 3.4653
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.5991 - mae: 2.9243 - val_l